In [52]:
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
# Download required NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [53]:
# Initialize WordNet Lemmatizer for word normalization
lemmatizer = WordNetLemmatizer()
# Load intents from JSON file
intents = json.loads(open('intents.json').read())

In [54]:
# Extract unique tags from intents
tags = []
for entry in intents['intents']:
    tags.append(entry['tag'])
        

In [ ]:
tags

In [56]:
# Initialize lists to hold words, classes, and training data
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

# Tokenize patterns from intents and preprocess data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # Use extend to add words into word_list
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        

In [57]:
# Normalize words and remove ignored characters
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [58]:
classes = sorted(set(classes))

In [59]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [60]:
# Prepare training data
training = []
output_empty = [0] * len(classes)

In [61]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [62]:
# Shuffle and convert training data to numpy array
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Justi\AppData\Local\Temp\ipykernel_10340\4024136173.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [63]:
# Build the neural network model using Keras
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

In [64]:
from tensorflow.keras.optimizers import legacy

In [65]:
# Configure SGD optimizer
sgd = legacy.SGD(learning_rate=0.001,decay = 1e-6,momentum=0.9,nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

In [ ]:
# Train the model
model.fit(np.array(train_x),np.array(train_y), epochs = 500, batch_size = 2, verbose=1)

In [67]:
# Save the trained model
model.save('chatbot_model.model')

INFO:tensorflow:Assets written to: chatbot_model.model\assets


INFO:tensorflow:Assets written to: chatbot_model.model\assets


In [68]:
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

In [69]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [70]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.model')

In [71]:
def clean_up_sentence(sentence):
    sentence_words = nltk.wordpunct_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
                
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD=0.25
    result = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    
    result.sort(key = lambda x:x[1], reverse= True)
    return_list = []
    for r in result:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("bot running start")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

bot running start
1/1 [==============================] - 0s 36ms/step
I hope she's alright.
1/1 [==============================] - 0s 11ms/step
Just wondering.
1/1 [==============================] - 0s 12ms/step
Interesting background.
1/1 [==============================] - 0s 11ms/step
Greetings! What can I do for you?
1/1 [==============================] - 0s 11ms/step
Curious question.
1/1 [==============================] - 0s 12ms/step
Let's work together.
1/1 [==============================] - 0s 12ms/step
While I don't have personal opinions, emerging technologies like AI and machine learning have transformative potential.
1/1 [==============================] - 0s 11ms/step
Boosting creativity involves exploring new experiences, engaging in brainstorming sessions, and stepping out of your comfort zone.
1/1 [==============================] - 0s 11ms/step
You got it! If you need anything else, feel free to ask.


In [1]:
import sys
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextBrowser, QLineEdit, QPushButton, QVBoxLayout, QWidget

filename = 'intents.json'

class ChatBotGUI(QMainWindow):
    def __init__(self):
        super().__init__()

        self.initUI()

        self.lemmatizer = WordNetLemmatizer()
        self.intents = json.loads(open(filename).read())

        self.words = pickle.load(open('words.pkl', 'rb'))
        self.classes = pickle.load(open('classes.pkl', 'rb'))
        self.model = load_model('chatbot_model.model')

    def initUI(self):
        self.setGeometry(100, 100, 400, 600)
        self.setWindowTitle('ChatBot GUI')

        self.text_browser = QTextBrowser(self)
        self.user_input = QLineEdit(self)
        self.send_button = QPushButton('Send', self)

        layout = QVBoxLayout()
        layout.addWidget(self.text_browser)
        layout.addWidget(self.user_input)
        layout.addWidget(self.send_button)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        self.send_button.clicked.connect(self.on_send_button_click)

        # Apply styles using QSS
        self.setStyleSheet("""
            QMainWindow {
                background-color: #f0f0f0;
            }
            QTextBrowser {
                background-color: white;
                border: 1px solid #ccc;
                border-radius: 5px;
                padding: 10px;
            }
            QLineEdit {
                background-color: white;
                border: 1px solid #ccc;
                border-radius: 5px;
                padding: 5px;
            }
            QPushButton {
                background-color: #007BFF;
                color: white;
                border: none;
                border-radius: 5px;
                padding: 5px 10px;
            }
            QPushButton:hover {
                background-color: #0056b3;
            }
        """)

    def clean_up_sentence(self, sentence):
        sentence_words = nltk.wordpunct_tokenize(sentence)
        sentence_words = [self.lemmatizer.lemmatize(word) for word in sentence_words]
        return sentence_words

    def bag_of_words(self, sentence):
        sentence_words = self.clean_up_sentence(sentence)
        bag = [0] * len(self.words)
        for w in sentence_words:
            for i, word in enumerate(self.words):
                if word == w:
                    bag[i] = 1
        return np.array(bag)

    def predict_class(self, sentence):
        bow = self.bag_of_words(sentence)
        res = self.model.predict(np.array([bow]))[0]
        ERROR_THRESHOLD = 0.25
        result = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

        result.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in result:
            return_list.append({'intent': self.classes[r[0]], 'probability': str(r[1])})
        return return_list

    def get_response(self, intents_list, intents_json):
        tag = intents_list[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result

    def on_send_button_click(self):
        user_message = self.user_input.text()
        self.user_input.clear()

        ints = self.predict_class(user_message)
        response = self.get_response(ints, self.intents)

        self.text_browser.append("<span style='color: blue;'>You: </span>" + user_message)
        self.text_browser.append("<span style='color: green;'>Bot: </span>" + response)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    gui = ChatBotGUI()
    gui.show()
    sys.exit(app.exec_())


1/1 [==============================] - 0s 12ms/step


SystemExit: 0

C:\Users\Justi\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
